<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/GeometryBasedDecisions/code/Gorilla2BIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gorilla Info: <br>
Project: GeometryBasedDecisions <br>
Account: Roberto Vargas (robertov@andrew.cmu.edu)

In [1]:
#@title Install Dependancies
import pandas as pd
import numpy as np
import re

In [2]:
#@title Utility Functions

def convert_to_float(x):
    if isinstance(x, list):
        return float(x[0]) # assuming there is only one value in the list
    else:
        return x

In [3]:
#@title Load Data

#QUESTIONAIRRES:
#Demographics
url = 'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/refs/heads/main/GeometryBasedDecisions/data/raw/raw_demographic.csv'
df_demographic = pd.read_csv(url)

#News Consumption
url = 'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/refs/heads/main/GeometryBasedDecisions/data/raw/raw_questionnaire_NewsConsumption.csv'
df_News = pd.read_csv(url)

#Concept Engagement Pt1
url = 'https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/refs/heads/main/GeometryBasedDecisions/data/raw/raw_task_SocConEngagePt1.csv'
df_ConEngPt1 = pd.read_csv(url)

#Concept Engagement Pt23
url = 'https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/refs/heads/main/GeometryBasedDecisions/data/raw/raw_questionnaire_SocConEngagePt23.csv'
df_ConEngPt23 = pd.read_csv(url)

#PurposeQ
url = 'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/refs/heads/main/GeometryBasedDecisions/data/raw/raw_questionnaire_PurposeQ.csv'
df_PurposeQ = pd.read_csv(url)

#TASKS:
#PRaM
url = 'https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/refs/heads/main/GeometryBasedDecisions/data/raw/raw_task_PRaM.csv.gz'
df_taskPRaM = pd.read_csv(url, compression='gzip')


<ipython-input-3-8429c8b7a8a7>:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_demographic = pd.read_csv(url)
<ipython-input-3-8429c8b7a8a7>:14: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ConEngPt1 = pd.read_csv(url)
<ipython-input-3-8429c8b7a8a7>:27: DtypeWarning: Columns (0,36,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_taskPRaM = pd.read_csv(url, compression='gzip')


In [4]:
df_ConEngPt1

,Event Index,UTC Timestamp,UTC Date and Time,Local Timestamp,Local Timezone,Local Date and Time,Experiment ID,Experiment Version,Tree Node Key,Repeat Key,...,Component Name,Object Name,Object Number,Object ID,Spreadsheet: display,Spreadsheet: Instituion,Spreadsheet: Question,Spreadsheet: randomize_trial,Spreadsheet: Answer,Manipulation: Spreadsheet
0,1,1.729268e+12,18/10/2024 16:16:39,1.729254e+12,-4.0,18/10/2024 08:16:39,187086.0,12.0,task-m7p7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.729268e+12,18/10/2024 16:16:41,1.729254e+12,-4.0,18/10/2024 08:16:41,187086.0,12.0,task-m7p7,NaN,...,Click To Continue,Continue Button,2.0,object-4,Instruction,NaN,Please read the following statements and selec...,NaN,NaN,NaN
2,3,1.729268e+12,18/10/2024 16:16:44,1.729254e+12,-4.0,18/10/2024 08:16:44,187086.0,12.0,task-m7p7,NaN,...,Click Response,Agree,3.0,object-10,Trial,Science,I believe that science as a process results in...,7.0,NaN,NaN
3,4,1.729268e+12,18/10/2024 16:16:46,1.729254e+12,-4.0,18/10/2024 08:16:46,187086.0,12.0,task-m7p7,NaN,...,Click Response,Agree,3.0,object-10,Trial,Science,I believe that science often conflicts with re...,7.0,NaN,NaN
4,5,1.729268e+12,18/10/2024 16:16:48,1.729254e+12,-4.0,18/10/2024 08:16:48,187086.0,12.0,task-m7p7,NaN,...,Click Response,Agree,3.0,object-10,Trial,Science,I believe that science is a driving force for ...,7.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25757,40,1.729291e+12,18/10/2024 22:31:08,1.729276e+12,-4.0,18/10/2024 14:31:08,187086.0,12.0,task-m7p7,NaN,...,Click Response,Agree,3.0,object-10,Trial,Firefighters,I believe I am safer with a firefighter statio...,3.0,NaN,NaN
25758,41,1.729291e+12,18/10/2024 22:31:10,1.729276e+12,-4.0,18/10/2024 14:31:10,187086.0,12.0,task-m7p7,NaN,...,Click Response,Agree,3.0,object-10,Trial,Firefighters,I would call firefighters for emergency assist...,3.0,NaN,NaN
25759,42,1.729291e+12,18/10/2024 22:31:11,1.729276e+12,-4.0,18/10/2024 14:31:11,187086.0,12.0,task-m7p7,NaN,...,Click Response,Agree,3.0,object-10,Trial,Firefighters,I believe that fire departments prioritize my ...,3.0,NaN,NaN
25760,43,1.729291e+12,18/10/2024 22:31:11,1.729276e+12,-4.0,18/10/2024 14:31:11,187086.0,12.0,task-m7p7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Isolate relevant columns: Demographics
include = ['Participant Private ID', 'Question', 'Key', 'Response']
df_demographic.drop(columns=df_demographic.columns.difference(include), inplace=True)
mask = df_demographic['Response'].str.contains('BEGIN', na=True) | df_demographic['Response'].str.contains('END', na=True)
df_demographic = df_demographic[~mask]

#Isolate relevant columns: News Consumption
include = ['Participant Private ID', 'Question', 'Key', 'Response']
df_News.drop(columns=df_News.columns.difference(include), inplace=True)
mask = df_News['Response'].str.contains('BEGIN', na=True) | df_News['Response'].str.contains('END', na=True)
df_News = df_News[~mask]

#Isolate relevant columns: Concept Engagement Pt1
include = ['Participant Private ID', 'Spreadsheet: display', 'Object Name', 'Spreadsheet: Instituion', 'Spreadsheet: Question']
df_ConEngPt1.drop(columns=df_ConEngPt1.columns.difference(include), inplace=True)
mask = df_ConEngPt1['Spreadsheet: display'].str.contains('Trial', na=True)
df_ConEngPt1 = df_ConEngPt1[mask]

#Isolate relevant columns: Concept Engagement Pt23
include = ['Participant Private ID', 'Key', 'Response']
df_ConEngPt23.drop(columns=df_ConEngPt23.columns.difference(include), inplace=True)
mask = df_ConEngPt23['Key'].str.contains('value', na=True)
df_ConEngPt23 = df_ConEngPt23[mask]

#Isolate relevant columns: PurposeQ [DEBUG RELEVANT COLUMNS get long-form spreadsheet]
include = ['Participant Private ID', 'Question', 'Key', 'Response']
df_PurposeQ.drop(columns=df_PurposeQ.columns.difference(include), inplace=True)
mask = df_PurposeQ['Response'].str.contains('BEGIN', na=True) | df_PurposeQ['Response'].str.contains('END', na=True)
df_PurposeQ = df_PurposeQ[~mask]

#Isolate relevant columns: PRaM
include = ['Participant Private ID', 'Zone Name', 'conceptA', 'conceptB', 'Response']
df_taskPRaM.drop(columns=df_taskPRaM.columns.difference(include), inplace=True)
mask = df_taskPRaM['Zone Name'].str.contains('Likert', na=True)
df_taskPRaM = df_taskPRaM[mask]
df_taskPRaM = df_taskPRaM.dropna()
df_taskPRaM.drop(columns=['Zone Name'], inplace=True)
df_taskPRaM['Pair'] = df_taskPRaM['conceptA'] + '-' + df_taskPRaM['conceptB']
df_taskPRaM.drop(columns=['conceptA', 'conceptB'], inplace=True)


KeyError: 'Response'

In [6]:
df_ConEngPt1

,Participant Private ID,Object Name,Spreadsheet: display,Spreadsheet: Instituion,Spreadsheet: Question
0,11889642.0,NaN,NaN,NaN,NaN
2,11889642.0,Agree,Trial,Science,I believe that science as a process results in...
3,11889642.0,Agree,Trial,Science,I believe that science often conflicts with re...
4,11889642.0,Agree,Trial,Science,I believe that science is a driving force for ...
5,11889642.0,Agree,Trial,Science,"If I could, I would vote for policies that inc..."
...,...,...,...,...,...
25757,11892003.0,Agree,Trial,Firefighters,I believe I am safer with a firefighter statio...
25758,11892003.0,Agree,Trial,Firefighters,I would call firefighters for emergency assist...
25759,11892003.0,Agree,Trial,Firefighters,I believe that fire departments prioritize my ...
25760,11892003.0,NaN,NaN,NaN,NaN


In [ ]:
#Isolate Pivot dataframes
df_demographic = df_demographic.pivot(index = 'Participant Private ID', columns = ['Question', 'Key'], values = ['Response']) #Demographics

df_News = df_News.pivot(index = 'Participant Private ID', columns = ['Question', 'Key'], values = ['Response']) #News Consumption

df_ConEngPt1 = df_ConEngPt1.pivot(index = 'Participant Private ID', columns = ['Question', 'Key'], values = ['Response']) #News Consumption

df_ConEngPt23 = df_ConEngPt23.pivot(index = 'Participant Private ID', columns = ['Question', 'Key'], values = ['Response']) #News Consumption

df_PurposeQ = df_PurposeQ.pivot(index = 'Participant Private ID', columns = ['Question', 'Key'], values = ['Response']) #News Consumption

#TASKS:
df_taskPRaM = df_taskPRaM.groupby(['Participant Private ID', 'Pair'])['Response'].agg(list).reset_index()
df_taskPRaM = df_taskPRaM.pivot(index='Participant Private ID', columns='Pair', values='Response')
df_taskPRaM = df_taskPRaM.applymap(convert_to_float)


<ipython-input-79-efd625e67af2>:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_taskPRaM = df_taskPRaM.applymap(convert_to_float)


In [ ]:
#Convert all files to csv
df_demographic.to_csv('demographic.csv')
df_News.to_csv('df_News.csv')
df_ConEngPt1.to_csv('df_ConEngPt1.csv')
df_ConEngPt23.to_csv('df_ConEngPt23.csv')
df_PurposeQ.to_csv('df_PurposeQ.csv')
df_taskPRaM.to_csv('df_taskPRaM.csv')

In [ ]:
#Offline Processing is as follows:

#Remove nested columns
#Rename column headers to be more intuitive
#News: Added AllSides coding (10/23)